In [ ]:
import os
import requests
import utils
from lol_backend_service import RiotAPI
from constants import Clusters
from dotenv import load_dotenv

load_dotenv()

In [ ]:
riot_api = RiotAPI(os.getenv("PERSONAL_API_KEY"))

### Get PUUID

In [ ]:
player_puuid = riot_api.get_puuid_by_riot_id(os.getenv("PERSONAL_GAME_NAME"), os.getenv("PERSONAL_TAGLINE"))
player_puuid

### Get Free Champion Rotation

In [ ]:
curr_free_champ = utils.get_champion_mapping(riot_api.get_free_champ_rotation('PH2'))
curr_free_champ

### Get Match Data for analysis

In [ ]:
match_data = riot_api.get_match_data("SEA", os.getenv("TEST_MATCH_ID"))
match_data